# Orbital Elements to Cartesian Coordinates

In [1]:
# Library imports
import tensorflow as tf
import rebound
import numpy as np
import matplotlib.pyplot as plt
import time

# Aliases
keras = tf.keras

In [2]:
# Local imports
from utils import load_vartbl, save_vartbl, plot_style
from tf_utils import gpu_grow_memory
# from tf_utils import Identity
from r2b import VectorError
from orbital_element import make_data_orb_elt, make_dataset_elt_to_cfg, make_dataset_cfg_to_elt
from orbital_element import OrbitalElementToConfig, make_model_elt_to_cfg
from orbital_element import ConfigToOrbitalElement, make_model_cfg_to_elt
from orbital_element import MeanToEccentricAnomaly, MeanToTrueAnomaly

In [3]:
# Grow GPU memory (must be first operation in TF)
gpu_grow_memory()

In [4]:
# Plot style 
plot_style()

In [5]:
# Lightweight serialization
fname = '../pickle/orbital_element.pickle'
vartbl = load_vartbl(fname)

In [6]:
# Create small data set for orbital elements; dictionaries of numpy arrays
n = 100
a_min = 1.0
a_max = 2.0
e_max = 1.0
inc_max = np.pi/4.0
seed=42
elts, cart = make_data_orb_elt(n=n, a_min=a_min, a_max=a_max, e_max=e_max, inc_max=inc_max, seed=seed)

In [7]:
# Create a tensorflow Dataset instance in both directions
batch_size = 64
ds_e2c = make_dataset_elt_to_cfg(n=n, a_min=a_min, a_max=a_max, e_max=e_max, 
                                 inc_max=inc_max, seed=seed, batch_size=batch_size)
ds_c2e = make_dataset_cfg_to_elt(n=n, a_min=a_min, a_max=a_max, e_max=e_max, 
                                 inc_max=inc_max, seed=seed, batch_size=batch_size)

In [8]:
# Example batch
elts, cart = list(ds_e2c.take(1))[0]

# Unpack orbital elements
a = elts['a']
e = elts['e']
inc = elts['inc']
Omega = elts['Omega']
omega = elts['omega']
f = elts['f']
mu = elts['mu']

# Unpack cartesian coordinates
q = cart['q']
v = cart['v']

# Review shapes
print(f'Example batch sizes:')
print(f'a    = {a.shape}')
print(f'e    = {e.shape}')
print(f'inc  = {inc.shape}')
print(f'Omega= {Omega.shape}')
print(f'omega= {omega.shape}')
print(f'f    = {f.shape}')
print(f'mu   = {f.shape}')
print(f'q    = {q.shape}')
print(f'v    = {v.shape}')

Example batch sizes:
a    = (64,)
e    = (64,)
inc  = (64,)
Omega= (64,)
omega= (64,)
f    = (64,)
mu   = (64,)
q    = (64, 3)
v    = (64, 3)


In [9]:
# Run the layer on the batch of orbital elements
inputs_e2c = (a, e, inc, Omega, omega, f, mu)
cart_rec = OrbitalElementToConfig()(inputs_e2c)

In [10]:
# Create a model mapping orbital elements to cartesian coordinates
model_e2c = make_model_elt_to_cfg()

In [11]:
# Inputs to compile this model
optimizer = keras.optimizers.Adam(learning_rate=1.0E-3)

loss = {'q': VectorError(name='q_loss'),
        'v': VectorError(name='v_loss')}

metrics = None

loss_weights = {'q': 1.0,
                'v': 1.0}

In [12]:
# Compile the e2c model
model_e2c.compile(optimizer=optimizer, loss=loss, metrics=metrics, loss_weights=loss_weights)

In [13]:
# Verify that model matches rebound
model_e2c.evaluate(ds_e2c)

2/2 [==============================] - 0s 129ms/step - loss: 3.1243e-14 - q_loss: 2.0652e-14 - v_loss: 1.0591e-14


[3.1243021170678734e-14, 2.065208e-14, 1.0590941e-14]

In [14]:
# Summary of the model mapping orbital elements to position
# model_e2c.summary()

In [15]:
# Run the layer on the batch of orbital elements
qx = q[:,0]
qy = q[:,1]
qz = q[:,2]
vx = v[:,0]
vy = v[:,1]
vz = v[:,2]
inputs_c2e = (qx, qy, qz, vx, vy, vz, mu)
elt_rec = ConfigToOrbitalElement()(inputs_c2e)

In [16]:
# Create a model mapping cartesian coordinates to orbital elements
model_c2e = make_model_cfg_to_elt()

In [17]:
# Inputs to compile the c2e model
optimizer = keras.optimizers.Adam(learning_rate=1.0E-3)

loss = {'a': keras.losses.MeanSquaredError(),
        'e': keras.losses.MeanSquaredError(),
        'inc': keras.losses.MeanSquaredError(),
        'Omega': keras.losses.MeanSquaredError(),
        'omega': keras.losses.MeanSquaredError(),
        'f': keras.losses.MeanSquaredError(),
       }

metrics = None

loss_weights = {'a': 1.0,
                'e': 1.0,
                'inc': 1.0,
                'Omega': 1.0,
                'omega': 1.0,
                'f': 1.0}

In [18]:
# model_c2e.summary()

In [19]:
# Compile the c2e model
model_c2e.compile(optimizer=optimizer, loss=loss, metrics=metrics, loss_weights=loss_weights)

W0711 12:14:42.737786 140249643951936 training_utils.py:1237] Output M missing from loss dictionary. We assume this was done on purpose. The fit and evaluate APIs will not be expecting any data to be passed to M.
W0711 12:14:42.739298 140249643951936 training_utils.py:1237] Output N missing from loss dictionary. We assume this was done on purpose. The fit and evaluate APIs will not be expecting any data to be passed to N.


In [20]:
model_c2e.evaluate(ds_c2e)

2/2 [==============================] - 0s 207ms/step - loss: 2.8515e-12 - a_loss: 7.1528e-13 - e_loss: 6.3892e-15 - inc_loss: 7.3696e-13 - Omega_loss: 7.0521e-14 - omega_loss: 5.5364e-13 - f_loss: 7.6871e-13


[2.8515038637613732e-12,
 7.152797e-13,
 6.3892394e-15,
 7.369649e-13,
 7.0520555e-14,
 5.5363776e-13,
 7.6871175e-13]

In [ ]:
class MeanToEccentricAnomaly(keras.layers.Layer):
    """
    Convert the mean anomaly M to the eccentric anomly E given the eccentricity E.
    """
    def call(self, inputs):
        # Unpack inputs
        M, e = inputs
        
        # Move M to the inveral [0, 2pi]
        two_pi = tf.constant(2.0 * np.pi)
        M = tf.math.floormod(M, two_pi)

        # Initialize E; guess M when eccentricity is small, otherwise guess pi
        pi = tf.constant(np.pi)
        E = tf.where(condition= e < 0.8, x=M, y=pi)
        
        # Initial error; from Kepler's equation M = E - e sin(E)
        F = E - e * tf.sin(E) - M
        
        # Iterate to improve E; trial and error shows 8 usually enough for single precision convergence
        for i in range(10):
            # Only update where the error is above the tolerance
            condition = tf.greater(tf.abs(F), tf.constant(1.0E-16))
            E = tf.where(condition=condition, x=E - F / (1.0 - e * tf.cos(E)), y=E)
            F = tf.where(condition=condition, x=E - e * tf.sin(E) - M, y=F)
        
        # Put E in the range [0, 2pi]        
        return tf.math.floormod(E, two_pi)
        
    def get_config(self):
        return dict()

In [ ]:
class MeanToTrueAnomaly(keras.layers.Layer):
    """
    Convert the mean anomaly M to the true anomly f given the eccentricity E.
    """
    def call(self, inputs):
        # Unpack inputs
        M, e = inputs
        
        # Compute the eccentric anomaly E
        E = MeanToEccentricAnomaly()((M, e))
        
        # Compute the true anomaly from E
        return 2.0*tf.math.atan(tf.sqrt((1.0+e)/(1.0-e))*tf.math.tan(0.5*E))
        
    def get_config(self):
        return dict()

In [21]:
a, e, inc, Omega, omega, f, M, N = elt_rec

In [25]:
# Test the mean anomaly convervsion functions
E = MeanToEccentricAnomaly()((M, e))
f_rec = MeanToTrueAnomaly()((M, e))

# Compute the RMS error
f_err = f_rec - f
rms_err = np.sqrt(np.mean(f_err * f_err))
print(f'RMS error of true anomaly f from mean anomaly M:')
print(f'{rms_err:5.2E}')

RMS error of true anomaly f from mean anomaly M:
2.40E-06
